In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/data.yaml
/kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/valid/labels/IMG_20230131_114154_jpg.rf.3707afe672c849abe8453bf0c295b951.txt
/kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/valid/labels/maxresdefault-1-_jpg.rf.f045076e46a7996647a278e37c4c5242.txt
/kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/valid/labels/images-16-_jpeg_jpg.rf.d1a3bd1f873dad3aee21ee07185b30b3.txt
/kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/valid/labels/IMG_3621_block_2_2_png_jpg.rf.cc1d7f05cd8c316cb28b292f064de460.txt
/kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/valid/labels/good_weld_vid1929_jpeg_jpg.rf.333a5b032b03f427a0bc7b037dc5b49e.txt
/kaggle/inpu

In [8]:
import os
import random
import torch
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection import maskrcnn_resnet50_fpn
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [9]:
CLASS_NAMES = ['Bad Welding', 'Crack', 'Excess Reinforcement', 'Good Welding', 'Porosity', 'Spatters']

class WeldInstanceDataset(Dataset):
    def __init__(self, root_dir, transforms=None, img_size=(640, 640)):
        self.root_dir = root_dir
        self.image_dir = os.path.join(root_dir, "images")
        self.label_dir = os.path.join(root_dir, "labels")
        self.image_filenames = [f for f in os.listdir(self.image_dir) if f.endswith('.jpg') or f.endswith('.png')]
        self.transforms = transforms
        self.img_size = img_size

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_filename = self.image_filenames[idx]
        img_path = os.path.join(self.image_dir, img_filename)
        label_path = os.path.join(self.label_dir, img_filename.rsplit('.', 1)[0] + ".txt")

        image = Image.open(img_path).convert("RGB")
        image = image.resize(self.img_size)
        img_np = np.array(image)
        height, width = self.img_size

        boxes = []
        masks = []
        labels = []
        with open(label_path, 'r') as f:
            for line in f:
                class_id, x_c, y_c, w, h = map(float, line.strip().split())
                x_c, y_c, w, h = x_c * width, y_c * height, w * width, h * height
                x1 = int(x_c - w / 2)
                y1 = int(y_c - h / 2)
                x2 = int(x_c + w / 2)
                y2 = int(y_c + h / 2)
                boxes.append([x1, y1, x2, y2])

                mask = np.zeros((height, width), dtype=np.uint8)
                mask[y1:y2, x1:x2] = 1
                masks.append(mask)
                labels.append(int(class_id) + 1)  # background is 0

        target = {
            "boxes": torch.as_tensor(boxes, dtype=torch.float32),
            "labels": torch.as_tensor(labels, dtype=torch.int64),
            "masks": torch.as_tensor(np.stack(masks), dtype=torch.uint8),
            "image_id": torch.tensor([idx])
        }

        if self.transforms:
            image = self.transforms(image)

        return image, target

In [10]:
def visualize_dataset_sample(dataset, num_samples=3):
    for i in range(num_samples):
        image, target = dataset[random.randint(0, len(dataset)-1)]
        img_np = image.permute(1, 2, 0).numpy() * 255
        img_np = img_np.astype(np.uint8)

        masks = target['masks'].numpy()
        labels = target['labels'].numpy()

        for j, mask in enumerate(masks):
            color = np.random.randint(0, 255, (3,), dtype=np.uint8)
            img_np[mask == 1] = img_np[mask == 1] * 0.5 + color * 0.5
            x1, y1, x2, y2 = target['boxes'][j].int().numpy()
            cv2.rectangle(img_np, (x1, y1), (x2, y2), color.tolist(), 2)
            cv2.putText(img_np, CLASS_NAMES[labels[j]-1], (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color.tolist(), 2)

        plt.figure(figsize=(8, 8))
        plt.imshow(img_np)
        plt.axis('off')
        plt.title("Sample with Masks and Labels")
        plt.show()

transforms = T.Compose([
    T.ToTensor(),
])

dataset = WeldInstanceDataset("/kaggle/input/weld-quality-inspection-instance-segmentation/Weld quality inspection - Segmentation/train", transforms=transforms)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# Visualize samples
visualize_dataset_sample(dataset, num_samples=3)

ValueError: too many values to unpack (expected 5)

In [12]:
dataset[10]

ValueError: too many values to unpack (expected 5)